In [2]:
from IPython.core.display import Markdown, display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import traceback
import numpy as np
import scipy.stats as ss
import yaml
import itertools
import sys
import os
import subprocess
from os import path
import pickle
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from sklearn.linear_model import LinearRegression, TheilSenRegressor, RANSACRegressor, HuberRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline

def printmd(*args):
    display(Markdown(' '.join(map(str, args))))

/tmp/ipykernel_13933/4167336305.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import Markdown, display, HTML


In [13]:
!rosrun local_planning_performance_modelling compute_metrics.py 

start : compute_metrics  11% session_2022-07-29_12-46-23_384597_run_000000000
start : compute_metrics  11% session_2022-07-29_12-56-08_037686_run_000000000
start : compute_metrics  11% session_2022-08-11_15-16-16_261512_run_000000001
start : compute_metrics  11% session_2022-08-11_15-38-35_112598_run_000000002
start : compute_metrics  11% session_2022-08-12_12-32-54_864279_run_000000000
start : compute_metrics  11% session_2022-08-12_12-34-05_726705_run_000000000
start : compute_metrics  11% session_2022-08-12_12-56-06_683404_run_000000000
start : compute_metrics  11% session_2022-08-12_13-15-32_973934_run_000000000
start : compute_metrics  11% session_2022-08-12_13-15-32_973934_run_000000001
cpu_time_and_max_memory: ps_snapshot files not found:
/home/emanuelevivani/ds/performance_modelling/output/test_local_planning/session_2022-08-12_12-32-54_864279_run_000000000/benchmark_data/ps_snapshots/ps_*.pkl 
trajectory_length: event navigation_goal_accepted not in events file:
/home/emanuele

In [16]:
df = pd.read_csv(path.expanduser("~/ds/performance_modelling/output/test_local_planning/results.csv"))
with open(path.expanduser("~/ds/performance_modelling/output/test_local_planning/results_info.yaml")) as results_info_file:
    results_info = yaml.safe_load(results_info_file)

# turn odometry_error into alpha_1..4
results_info['run_parameter_names'] += ['alpha_1', 'alpha_2', 'alpha_3', 'alpha_4']
results_info['run_parameter_names'].remove('odometry_error')
for i in range(0, 4):
    df[f'alpha_{i+1}'] = df['odometry_error'].apply(lambda x: eval(x)[i])
del df['odometry_error']

df.loc[df.robot_model == 'turtlebot3_waffle_performance_modelling', 'robot_model'] = 'turtle'
df.loc[df.robot_model == 'hunter2', 'robot_model'] = 'hunter'

run_parameters = [c for c in list(df.columns) if c in results_info['run_parameter_names']]
metrics_versions = [c for c in list(df.columns) if '_version' in c]
everything_else = ['run_id', 'session_id', 'run_number']
metrics = [c for c in df.columns if c not in metrics_versions + run_parameters + everything_else]
metrics_and_versions = [c for c in list(df.columns) if '_version' in c or c in metrics]

cpu_time_metrics = [c for c in metrics if 'cpu_time' in c]
max_memory_metrics = [c for c in metrics if 'max_memory' in c]
odometry_metrics = [c for c in metrics if 'odometry_error_alpha_' in c]
odometry_mean_metrics = [c for c in metrics if 'odometry_error_alpha_' in c and '_mean' in c]
odometry_std_metrics = [c for c in metrics if 'odometry_error_alpha_' in c and '_std' in c]
odometry_parameters = [f'alpha_{i}' for i in range(1, 5)]
localization_update_metrics = [c for c in metrics if 'localization_update_' in c]
localization_update_mean_metrics = [c for c in metrics if 'localization_update_' in c and '_mean' in c]
localization_update_translation_mean_metrics = [c for c in metrics if 'localization_update_' in c and '_translation_' in c and '_mean' in c]
navigation_metrics = [c for c in metrics if c in ['trajectory_length', 'execution_time', 'success_rate']]

# add metrics from existing ones
df['mean_velocity'] = df['trajectory_length'] / df['execution_time']
navigation_metrics += ['mean_velocity']
metrics += ['mean_velocity']
metrics_and_versions += ['mean_velocity']

# add useful parameters
df['session_id'] =  df['run_id'].apply(lambda x:  x.split('_')[1]+'_'+x.split('_')[2]+'_'+x.split('_')[3])
df['run_number'] =  df['run_id'].apply(lambda x:  int(x.split('_')[5]))
df[max_memory_metrics] = df[max_memory_metrics]/1024**2
#df.max_steering_angle_deg = df.max_steering_angle_deg.fillna(90)

printmd("## Number of Runs")
printmd(f"    {len(df.run_id.unique())}")

printmd("## Run Parameters")
for name in [run_parameter_name for run_parameter_name in run_parameters if 'localization_generator_' not in run_parameter_name]:
    values = list(df[name].unique())
    printmd(f"    {name:<70}", sorted(values))

printmd("## Metrics")
for name in metrics_and_versions:
    if name in metrics_versions:
        if len(df[name].unique()) == 1:
            printmd(f"    {name:<70} {sorted(df[name].unique())}")
        else:
            printmd(f"<code><font style='background-color:yellow;font-family:monospace'>{name:<70}{sorted(df[name].unique())} </font></code> ⚠️")
    else:
        printmd(f"    {name:<70} min: {df[name].min(skipna=True):10.4g} {'avg':>15}: {df[name].mean(skipna=True):10.4g} {'max':>15}: {df[name].max(skipna=True):10.4g}]")

## Number of Runs

    9

## Run Parameters

    amcl_alpha_factor                                                      [0]

    environment_name                                                       ['airlab']

...

    average_translation_acceleration                                       min:    0.05456             avg:     0.0829             max:      0.146]

    average_rotation_acceleration                                          min:    0.06958             avg:     0.1948             max:     0.3805]

    translation_rotation_acceleration_product                              min:   0.009935             avg:    0.02744             max:    0.06592]

    motion_characteristics_version                                         [3]

    mean_velocity                                                          min:    0.04446             avg:    0.09596             max:     0.1851]

In [ ]:
df[['run_id', 'robot_model', 'collision_rate']]